## Содержание
<a id='Содержание'></a>

[Импорты и загрузка данных](#Импорты_и_загрузка)

[Предотработка](#Предотработка)

[Разделение на выборки](#Разделение_на_выборки)

[Тестирование моделей](#Тестирование_моделей)

[Проверка качества модели](#Проверка_качества_модели)

[Общий вывод](#Общий_вывод)

## Mobile Plan Recommendations

The goal of this project was to provide recommendations to clients of a telecom company, who have yet to switch to one of the two new mobile plans introduced by the company.

The data used in this project is the same data that was used for comparing the profitability of the two plans in the project Analysis of Telecom Service Usage (project 4). As such, the data is already cleaned/prepared. The end goal was different: train and test a ML classification model that can later be used for recommending plans to other clients based on their usage of the company's older plans.

This project was an introduction to ML models. If I were going to do it again or to edit it, I would change certain things; most notably, I would change the way I tested model parameters. Another example would be to compare the results of my chosen model to the results of a dummy model.

### Импорты и загрузка данных
<a id='Импорты_и_загрузка'></a>

In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

Открытие файла и изучение данных

In [3]:
try:
    user_data = pd.read_csv('datasets/users_behavior.csv')
except FileNotFoundError as e:
    print(e)
    user_data = pd.read_csv('/datasets/users_behavior.csv')
    
display(user_data.head())
user_data.info()
user_data.describe().T

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


### Предотработка
<a id='Предотработка'></a>

Проверка возможной коррелации между данными

In [4]:
user_data.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


Поскольку наблюдается высокая коррелация между 'calls' и 'minutes', стоит исключить один из этих столбцов до анализа. Высокая коррелация между двумя признаками может путать функции машинного обучения и искажить результаты. Поскольку 'minutes' содержит более точную информацию чем 'calls', здесь лучше убрать 'calls'.

Устранение столбца 'calls'

In [5]:
user_data = user_data.drop('calls', axis=1)

### Разделение на выборки
<a id='Разделение_на_выборки'></a>

Разделение данных на три выборки: 
- обучающую (40% целокупности данных)
- валидационную (30% целокупности данных)
- тестовую (30% целокупности данных)

In [6]:
train_data, valid_and_test = train_test_split(user_data, test_size=0.4, random_state=42)
valid_data, test_data = train_test_split(valid_and_test, test_size=0.5, random_state=42)

display(train_data, valid_data, test_data)

,minutes,messages,mb_used,is_ultra
2369,436.40,89.0,8495.06,1
2234,489.70,13.0,10671.53,0
1058,423.95,44.0,7826.03,0
118,215.41,0.0,6045.15,0
1024,709.40,17.0,16964.33,0
...,...,...,...,...
1095,454.02,35.0,15018.46,0
1130,465.96,12.0,14982.27,0
1294,280.44,2.0,13934.54,0
860,410.23,68.0,16006.55,0


,minutes,messages,mb_used,is_ultra
1198,185.22,0.0,3839.46,0
2918,445.00,100.0,10518.28,1
2615,296.04,38.0,22479.57,0
1745,421.16,5.0,15757.01,0
2573,419.72,0.0,43325.34,1
...,...,...,...,...
1993,244.64,0.0,41587.07,1
589,706.89,44.0,7800.95,0
1159,318.00,71.0,19221.34,0
3136,219.84,8.0,9502.52,0


,minutes,messages,mb_used,is_ultra
1545,166.51,77.0,12199.43,0
3173,430.76,84.0,26117.92,0
2290,510.85,0.0,20154.23,1
2645,568.92,60.0,18257.71,0
916,375.91,35.0,12388.40,0
...,...,...,...,...
1651,408.53,2.0,15346.04,0
842,476.37,76.0,13424.26,0
532,496.06,14.0,13850.84,1
456,593.77,0.0,11311.21,1


Создание переменные чтобы сохранить целевой признак отдельно от остальных

In [7]:
features_train = train_data.drop('is_ultra', axis=1)
target_train = train_data['is_ultra']
features_valid = valid_data.drop('is_ultra', axis=1)
target_valid = valid_data['is_ultra']
features_test = test_data.drop('is_ultra', axis=1)
target_test = test_data['is_ultra']

### Тестирование моделей
<a id='Тестирование_моделей'></a>

Тестирование разных моделей

In [8]:
best_model = None
best_result = 0
best_forest = 1
best_depth = 1
for trees in range(1, 11):
    for depth in range(1,9):
        model = RandomForestClassifier(random_state=42, max_depth=depth, n_estimators=trees)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_result = result
            best_model = model
            best_forest = trees
            best_depth = depth
        
print("Accuracy наилучшей модели на валидационной выборке:", best_result, "\nколичество деревев:", best_forest, '\n глубина:', best_depth)

Accuracy наилучшей модели на валидационной выборке: 0.8055987558320373 
количество деревев: 5 
 глубина: 8


In [9]:
# best_model = None
# best_result = 0
# for i in range(1,9):
#     model = DecisionTreeClassifier(max_depth=i, random_state=42)
#     model.fit(features_train, target_train)
#     predictions_valid = model.predict(features_valid)
#     result = model.score(features_valid, target_valid)
#     print("max_depth =", i, ": ", end='')
#     print(result)
#     if best_result < result:
#         best_result = result
#         best_model = model



In [10]:
# best_model = None
# best_result = 0
# best_forest = 1
# best_depth = 1
# for trees in range(1, 11):
#     for depth in range(1,9):
#         for samp in range(2,5):
#             for leaf in range(1,3):
#                 model = RandomForestClassifier(random_state=42, max_depth=depth, n_estimators=trees, 
#                                                min_samples_split=samp, min_samples_leaf=leaf)
#                 model.fit(features_train, target_train)
#                 result = model.score(features_valid, target_valid)
#                 if result > best_result:
#                     best_result = result
#                     best_model = model
#                     best_forest = trees
#                     best_depth = depth
#                     best_min_samp = samp
#                     best_min_leaf = leaf

# print("Accuracy наилучшей модели на валидационной выборке:", best_result, "\nколичество деревев:", best_forest,
#       '\nглубина:', best_depth, "\nминимальное количество признаков:", best_min_samp,
#       '\nминимальное количество признаков в одном листе:', best_min_leaf)

Тестирование моделей

- проверенные модели:
    - DecisionTreeClassifier
    - RandomForestClassifier
    - Поскольку целевой признак категориальный, логичнее использовать один из выше перечисленных моделей чем, например, LinearRegression или DecisionTreeRegressor
- каждая модель тестирована с разными гиперпараметрами
    - для этого применились for циклы
- оставил проверки невыбранных моделей в коде – они просто закомментированы 


Выбранная модель: RandomForestClassifier с 5 деревьями и глубиной 8

Точность моделей на валидационной выборке: 0.8055987558320373 

Получилось так, что почти все модели были достаточно точными для заданной задачи. Единственная модель, которая показывала точность меньше 75% была DecisionTreeClassifier с глубиной 1. Самая высокая точность была у RandomForestClassifier. Я проверял эту модель дважды: первый раз только меняя гиперпараметры max_depth и n_estimators, а второй меняя и эти, и min_samples_split и min_samples_leaf. Зато, не наблюдалось никакой разницы между результатами этих проверок. Поскольку не было разницы, я оставил более простую проверку потому, что питон её быстрее обрабатывает.

### Проверка качества модели
<a id='Проверка_качества_модели'></a>

Проверка качества модели на тестовой выборке

In [11]:
best_model.fit(features_train, target_train)
test_result = best_model.score(features_test, target_test)
print(f"Точность модели в предсказании целового признака на тестовой выборке: {test_result}")


Точность модели в предсказании целового признака на тестовой выборке: 0.8040435458786936


Модель оказалась достаточной качественной. На тестовой выборке, она предсказала целевой признак с точностью 80.4%

### Общий вывод
<a id='Общий_вывод'></a>

Функции scikit-learn, создавающие модели с помощью машинного обучения, хорошо работали на наших данных. Наш целевой признак был категоическим, поэтому приходилось использовать классификаторы а не функции, которые работают по принципу регрессии.

В конце концов, удалось создать модель, которая смогла предсказать целевой признак с точностью выше 80%.
